# Testing the Training and Scoring scripts locally

Create a data folder, and download the training/testing CIFAR data into that folder.

In [ ]:
%%bash
cd pytorch_image_classification && 
    mkdir data && cd data &&
    wget http://pjreddie.com/media/files/cifar.tgz &&
    tar xzf cifar.tgz

Run train0.ipynb to generate your model file (`model0`)

In [ ]:
%%bash
cd pytorch_image_classification &&
    ipython nbconvert --to python train0.ipynb &&
    python train0.py

We should now see our model file in the pytorch_image_classification directory:

In [ ]:
%%bash
ls pytorch_image_classification

To make sure our scoring script works, lets also test it. By default, it will look for a model file in `./`

In [ ]:
%%bash
cd pytorch_image_classification && 
    python score0.py

This should output a file that shows the predictions. Lets inspect `tail -10`

In [ ]:
%%bash
cd pytorch_image_classification && 
    cat scores_* | tail -20

# Setting up Azure Resources for Batch scoring at scale

The next steps will show how you set up all the Azure resources needed in order to run this Batch Scoring job at scale.

## Setting up Azure Batch AI with a file share

First, we need to create your cluster using the __azure/scripts/create_cluster.py__ script. (TODO - this should be expanded)

After running this command, go into the Azure portal and check that your cluster is successfully created.

In [ ]:
%%bash 
python azure/scripts/create_cluster.py

Now we need to upload the script and model files to the fileshare using the __azure/scripts/upload_files.py__ script. (TODO - this should be expanded)

After running this command, go into the Azure portal and make sure that the files are correctly uploaded.

In [ ]:
%%bash
python azure/scripts/upload_files.py --upload-data

Finally, lets test that our job is running correctly using the __azure/scripts/create_job.py__ script. (TODO - this should be expanded)

After running this command, go into the Azure portal and check that your job is successfully created.

In [ ]:
%%bash
python azure/scripts/create_job.py

### Setting up ACI to run the BatchAI Job

After we've tested that the BatchAI pieces work, we now want to build a docker container and check that we can run the BatchAI job from a docker container.

First we need to build the docker image using the Dockerfile which will upload all Azure utility python files as well as the __create_job.py__ file into the image.

In [ ]:
%%bash
cd azure &&
    sudo docker build -t bai_job .

Then we need to test that the BatchAI job can be executed from the docker image we just built. Because the __create_job.py__ file requires many environment variables, we will use the __docker_run.sh__ script to pass in all the required environment variables for the image to successfully run locally. 

In [ ]:
%%bash
cd azure &&
    source docker_run.sh -t bai_job 

Finally, we have to publish the image to Dockerhub (TODO - use ACR)

In [ ]:
%%bash
cd azure &&
    sudo docker tag bai_job <your-dockerhub-username>:bai_job &&
    sudo docker push <your-dockerhub-username>:bai_job 

## Setting up ACI with Logic Apps

TODO